# Add new labels to header for stage classification

In [1]:
from astropy.io import fits
import os

In [2]:
dataset_dir = "../processed_dataset"

## New mappings

In [3]:
category_mapping_1 = {
    "Elliptical/Lenticular": ["E", "E-S0", "E-SO", "S0", "S0-a"],
    "Spirals/Irregular": [
        "Sa", "Sab", "Sb", "SABa", "SABb",
        "Sbc", "SABc", "SBa", "SBb", "SBd", "SBbc",
        "Sc", "Scd", "Sd", "SBc", "SBcd", "SABd",
        "I", "IB", "IAB", "Sm", "SBm", "SABm", "SBab"
    ]
}

category_mapping_2 = {
    "Elliptical": ["E", "E-S0", "E-SO"],
    "Lenticular": ["S0", "S0-a"],
    "Irregulars": ["I", "IB", "IAB"],
    "Spirals/Barred Spirals": [
        "Sa", "Sab", "Sb", "Sbc", "Sc", "Sd", "Scd", "Sm",
        "SABa", "SABb", "SABc", "SABd", "SABm",
        "SBa", "SBab", "SBb", "SBbc",
        "SBc", "SBcd", "SBd" , "SBm"
    ]
}

category_mapping_3 = {
    "Elliptical": ["E", "E-S0", "E-SO"],
    "Lenticular": ["S0", "S0-a"],
    "Barred Spirals": ["SBa", "SBab", "SBb", "SBbc", "SBc", "SBcd", "SBd" , "SBm"],
    "Spirals": ["Sa", "Sab", "Sb", "Sbc", "Sc", "Sd", "Scd", "Sm", "SABa", "SABb", "SABc", "SABd", "SABm"],
    "Irregulars": ["I", "IB", "IAB"]
}

In [4]:
def get_category(g_type, mapping, mapping_name):
    for category, types in mapping.items():
        if g_type in types:
            return category
    print(f"Unknown type: {g_type}, mapping: {mapping_name}")

In [5]:
fits_files = [f for f in os.listdir(dataset_dir) if f.endswith(".fits")]

for fits_file in fits_files:
    file_path = os.path.join(dataset_dir, fits_file)
    with fits.open(file_path, mode="update") as hdul:
        header = hdul[0].header
        g_type = header.get("G_TYPE", "Unknown")
        
        header["CAT1bis"] = get_category(g_type, category_mapping_1, "Mapping 1")
        header["CAT2bis"] = get_category(g_type, category_mapping_2, "Mapping 2")
        header["CAT3bis"] = get_category(g_type, category_mapping_3, "Mapping 3")


        # Sauvegarde des modifications
        hdul.flush()

print("Catégories ajoutées à tous les fichiers FITS.")

Catégories ajoutées à tous les fichiers FITS.


In [6]:
first_fits_file = os.path.join(dataset_dir, fits_files[1])

data = fits.open(first_fits_file)
data[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                  171                                                  
NAXIS2  =                  171                                                  
NAXIS3  =                    3                                                  
G_NAME  = 'PGC0000963'                                                          
G_TYPE  = 'SABc    '                                                            
G_CAT   = 'Intermediate Spirals'                                                
FILTER_0= 'i       '                                                            
FILTER_1= 'r       '                                                            
FILTER_2= 'u       '                                                            
G_TYPE_8= 'Spiral Intermedia

In [7]:
for fits_file in fits_files[:100]:
    file_path = os.path.join(dataset_dir, fits_file)
    with fits.open(file_path) as hdul:
        header = hdul[0].header
        print(header.get("CAT2", "Unknown"))

Spirals/Irregulars
Spirals/Irregulars
Spirals/Irregulars
Barred Spirals
Barred Spirals
Spirals/Irregulars
Spirals/Irregulars
Lenticular
Lenticular
Spirals/Irregulars
Spirals/Irregulars
Spirals/Irregulars
Spirals/Irregulars
Barred Spirals
Spirals/Irregulars
Spirals/Irregulars
Barred Spirals
Barred Spirals
Barred Spirals
Barred Spirals
Spirals/Irregulars
Barred Spirals
Barred Spirals
Barred Spirals
Lenticular
Elliptical
Barred Spirals
Barred Spirals
Spirals/Irregulars
Lenticular
Spirals/Irregulars
Lenticular
Barred Spirals
Lenticular
Lenticular
Spirals/Irregulars
Barred Spirals
Spirals/Irregulars
Lenticular
Lenticular
Elliptical
Lenticular
Spirals/Irregulars
Spirals/Irregulars
Barred Spirals
Spirals/Irregulars
Barred Spirals
Spirals/Irregulars
Spirals/Irregulars
Spirals/Irregulars
Barred Spirals
Spirals/Irregulars
Elliptical
Spirals/Irregulars
Lenticular
Spirals/Irregulars
Elliptical
Barred Spirals
Spirals/Irregulars
Spirals/Irregulars
Spirals/Irregulars
Spirals/Irregulars
Spirals/Irregu

In [8]:
# Ouvrir le fichier FITS
with fits.open(first_fits_file) as hdul:
    # Accéder aux dimensions de l'image
    image_shape = hdul[0].data.shape
    print(f"La taille de l'image est: {image_shape}")

La taille de l'image est: (3, 171, 171)


In [1]:
import matplotlib.pyplot as plt

# Create a dictionary to store one image per category
images_per_category = {}

# Iterate over the FITS files to collect one image per category
for fits_file in fits_files:
    file_path = os.path.join(dataset_dir, fits_file)
    with fits.open(file_path) as hdul:
        header = hdul[0].header
        category = header.get("CAT3bis", "Unknown")
        
        if category not in images_per_category:
            images_per_category[category] = hdul[0].data

# Plot the images
fig, axes = plt.subplots(1, len(images_per_category), figsize=(15, 5))

for ax, (category, image) in zip(axes, images_per_category.items()):
    ax.imshow(image, cmap='gray')
    ax.set_title(category)
    ax.axis('off')

plt.tight_layout()
plt.show()

NameError: name 'fits_files' is not defined